In [23]:
import torch
import torch.nn.functional as F
from main import GPT2, GPTConfig
from transformers import GPT2Tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GPT2(GPTConfig())
model.load_state_dict(torch.load('epochs_25141.pth'))
model.to('cuda')
enc = GPT2Tokenizer.from_pretrained('gpt2')
enc

GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}
)

In [24]:
model.eval()
num_return_sequences = 4
max_length = 32
tokens = enc.encode("Hello I am pragateesh")
tokens = torch.tensor(tokens, dtype=torch.long)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)
xgen = tokens.to(device)
sample_rng = torch.Generator(device=device)
sample_rng.manual_seed(42)
while xgen.size(1) < max_length:
    with torch.no_grad():
        # with torch.autocast(device_type=device, dtype=torch.bfloat16):
        logits, _ = model(xgen)
        logits = logits[:, -1, :]
        probs = F.softmax(logits, dim=-1)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        ix = torch.multinomial(topk_probs, 1, generator=sample_rng)
        xcol = torch.gather(topk_indices, -1, ix)
        xgen = torch.cat((xgen, xcol), dim=1)
for i in range(num_return_sequences):
    tokens = xgen[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(f"\nsample {i}: \n{decoded}\n")



sample 0: 
Hello I am pragateeshark. My name was on that board!" This is a very similar approach of the first place game I made. This was a


sample 1: 
Hello I am pragateeshut!"

I thought that by now it appeared that I was at an end of my rope: "Hehehehe


sample 2: 
Hello I am pragateesh. I always said not to care how much they look like I was talking to you, but I am pragateesh. I


sample 3: 
Hello I am pragateesh. When I see a guy with a beard all the time, I say "Omg what's he doing here", and I

